In [2]:
import numpy as np
import pandas as pd
from ideas import intersection_over_union
from ideas import areas
box_columns = ['Xmin','Ymin','Xmax','Ymax']

In [3]:
t_d = pd.read_csv('train_data.csv')
t_d['pxs'] = (t_d['Xmax'] - t_d['Xmin'])*(t_d['Ymax'] - t_d['Ymin'])
pxs = t_d.groupby("itemId")['pxs'].mean()
t_d = t_d.merge(pxs, on=["itemId"])
t_d['pxs_ratio'] = t_d['pxs_x'] / t_d['pxs_y']
train_answers = pd.read_csv('train_answers.csv')
ids = list(set(t_d['itemId']))
users_rating = pd.read_csv("users_rating.csv")
users_rating = users_rating.set_index('userId')

In [6]:
t_d = pd.read_csv('train_data.csv')
t_a = pd.read_csv('train_answers.csv')
t_w_a = t_d.merge(t_a, on=["itemId"])
t_w_a['XminDifPer'] = (t_w_a['Xmin'] - t_w_a['Xmin_true'])/(t_w_a['Xmax_true'] - t_w_a['Xmin_true'])
t_w_a['XmaxDifPer'] = (t_w_a['Xmax'] - t_w_a['Xmax_true'])/(t_w_a['Xmax_true'] - t_w_a['Xmin_true'])
t_w_a['YminDifPer'] = (t_w_a['Ymin'] - t_w_a['Ymin_true'])/(t_w_a['Ymax_true'] - t_w_a['Ymin_true'])
t_w_a['YmaxDifPer'] = (t_w_a['Ymax'] - t_w_a['Ymax_true'])/(t_w_a['Ymax_true'] - t_w_a['Ymin_true'])

In [7]:
t_w_a.groupby("userId")['XminDifPer', 'XmaxDifPer', 'YminDifPer', 'YmaxDifPer'].mean().to_csv('users_adj.csv', header=True)

In [8]:
t_w_a['iou'] = t_w_a[['Xmin','Ymin', 'Xmax', 'Ymax', 'Xmin_true',\
      'Ymin_true', 'Xmax_true','Ymax_true']].apply(intersection_over_union, axis=1)
t_w_a.groupby("userId")['iou'].mean().to_csv('users_rating.csv', header=True)

In [ ]:
t_w_a = t_d.merge(max_result, on=["itemId"])

In [10]:
users_rating = pd.read_csv("users_rating.csv")
users_rating = users_rating.set_index('userId')

users_adj = pd.read_csv("users_adj.csv")
users_adj = users_adj.set_index('userId')

quorum = pd.DataFrame(np.zeros((len(ids), 5), dtype=int), index=ids, columns=box_columns + ['itemId'])
for i in ids:    
    uids = t_d[t_d['itemId'] == i]['userId']
    uid = users_rating.loc[uids].idxmax()['iou']
    ti = t_d.loc[t_d['itemId'] == i].loc[t_d['userId'] == uid][box_columns].max()
    uajd = users_adj.loc[uid]
    l = ti['Xmax'] - ti['Xmin']
    w = ti['Ymax'] - ti['Ymin']
    for y in box_columns:        
        t = l if "X" in y else w
        c = 1 if "min" in y else -1
        quorum.loc[i][y] = ti[y] + c * t * uajd[y + 'DifPer']
    quorum.loc[i]['itemId'] = i    
data = quorum.merge(train_answers, on=['itemId'])
data["iou"] = data[['Xmin','Ymin', 'Xmax', 'Ymax', 'Xmin_true',\
      'Ymin_true', 'Xmax_true','Ymax_true']].apply(intersection_over_union, axis=1)
print(data["iou"].mean())    

0.5395903589107386
